# Optimisation de Portefeuille avec le Modèle de Stock Picking

Ce notebook montre comment utiliser le module d'optimisation de portefeuille pour construire un portefeuille optimisé basé sur les scores du modèle de stock picking.

## Configuration et importations

In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

# Ajouter le répertoire parent au path pour accéder aux modules du projet
sys.path.append('..')

from src.data.collectors import YahooFinanceCollector
from src.data.preprocessors import PriceDataPreprocessor, FundamentalDataPreprocessor
from src.models.scoring import FundamentalScorer, TechnicalScorer, MultifactorScorer
from src.models.portfolio_optimization import PortfolioOptimizer

# Configuration du style des graphiques
plt.style.use('seaborn-darkgrid')
sns.set(font_scale=1.2)
plt.rcParams['figure.figsize'] = (14, 8)

## Collecte de données pour les actions

In [ ]:
# Définir les tickers à analyser (un univers plus large pour l'optimisation de portefeuille)
tickers = [
    # Technologies
    'AAPL', 'MSFT', 'GOOGL', 'AMZN', 'META', 'NVDA', 'TSLA', 'ADBE', 'CRM', 'INTC',
    # Finance
    'JPM', 'BAC', 'WFC', 'GS', 'MS', 'C', 'AXP', 'V', 'MA', 'BLK',
    # Santé
    'JNJ', 'PFE', 'MRK', 'UNH', 'ABBV', 'LLY', 'ABT', 'TMO', 'DHR', 'MDT',
    # Consommation
    'PG', 'KO', 'PEP', 'WMT', 'COST', 'HD', 'MCD', 'NKE', 'SBUX', 'DIS'
]

# Créer les répertoires de données si nécessaire
os.makedirs('../data/raw', exist_ok=True)
os.makedirs('../data/processed', exist_ok=True)
os.makedirs('../data/results', exist_ok=True)

# Définir la période d'analyse
end_date = datetime.now()
start_date = end_date - timedelta(days=365 * 3)  # 3 ans de données

# Initialiser le collecteur de données
collector = YahooFinanceCollector(output_dir='../data/raw')

# Récupérer les données de prix
print(f"Collecte des données de prix pour {len(tickers)} actions...")
price_data = collector.get_stock_data(
    tickers + ['^GSPC'],  # Inclure le S&P 500 comme référence
    start_date=start_date.strftime('%Y-%m-%d'),
    end_date=end_date.strftime('%Y-%m-%d')
)

# Récupérer les données fondamentales
print("Collecte des données fondamentales...")
fundamental_data = {}
for ticker in tickers:
    fundamental_data[ticker] = collector.get_fundamentals([ticker])

## Prétraitement des données

In [ ]:
# Initialiser les préprocesseurs
price_preprocessor = PriceDataPreprocessor(
    input_dir='../data/raw',
    output_dir='../data/processed'
)

fundamental_preprocessor = FundamentalDataPreprocessor(
    input_dir='../data/raw',
    output_dir='../data/processed'
)

# Prétraiter les données de prix
print("Prétraitement des données de prix...")
processed_price_data = {}
for ticker in tickers + ['^GSPC']:
    if ticker in price_data:
        # Trouver le fichier correspondant
        import glob
        files = glob.glob(f"../data/raw/{ticker}_*.csv")
        if files:
            filename = os.path.basename(files[0])
            processed_price_data[ticker] = price_preprocessor.preprocess(filename)

# Prétraiter les données fondamentales
print("Prétraitement des données fondamentales...")
processed_fundamental_data = {}
for ticker in tickers:
    if ticker in fundamental_data:
        # Vérifier que les fichiers existent
        income_file = f"{ticker}_income_stmt.csv"
        balance_file = f"{ticker}_balance_sheet.csv"
        cash_flow_file = f"{ticker}_cash_flow.csv"
        
        if os.path.exists(f"../data/raw/{income_file}") and os.path.exists(f"../data/raw/{balance_file}"):
            processed_fundamental_data[ticker] = fundamental_preprocessor.preprocess_financials(
                income_file,
                balance_file,
                cash_flow_file if os.path.exists(f"../data/raw/{cash_flow_file}") else None
            )

## Scoring des actions

In [ ]:
# Initialiser les scorers
fundamental_scorer = FundamentalScorer(output_dir='../data/results')
technical_scorer = TechnicalScorer(output_dir='../data/results')
multifactor_scorer = MultifactorScorer(
    output_dir='../data/results',
    factor_weights={'fundamental': 0.7, 'technical': 0.3}  # Pondération personnalisée
)

# Calculer les scores fondamentaux
print("Calcul des scores fondamentaux...")
fundamental_scores = fundamental_scorer.score_stocks(processed_fundamental_data)

# Calculer les scores techniques
print("Calcul des scores techniques...")
technical_scores = technical_scorer.score_stocks(
    processed_price_data, 
    processed_price_data.get('^GSPC')  # Utiliser le S&P 500 comme référence
)

# Calculer les scores multifactoriels
print("Calcul des scores multifactoriels...")
multifactor_scores = multifactor_scorer.score_stocks(
    processed_fundamental_data,
    processed_price_data,
    None,  # Pas de données qualitatives pour cet exemple
    processed_price_data.get('^GSPC')
)

# Afficher les 10 meilleures actions selon le score multifactoriel
print("\nTop 10 des actions selon le score multifactoriel:")
display(multifactor_scores.sort_values('overall_score', ascending=False).head(10))

## Optimisation de portefeuille basée sur le modèle de Markowitz

In [ ]:
# Initialiser l'optimiseur de portefeuille
optimizer = PortfolioOptimizer(output_dir='../data/results')

# Optimisation basée sur Markowitz (Mean-Variance)
print("Optimisation selon le modèle de Markowitz...")
markowitz_portfolio = optimizer.optimize(
    processed_price_data,
    multifactor_scores,
    method="markowitz",
    risk_free_rate=0.025,  # Taux sans risque annualisé (2.5%)
    target_return=None,    # Maximisation du ratio de Sharpe
    max_weight=0.15,       # Pas plus de 15% dans une seule action
    min_weight=0.01        # Au moins 1% si incluse dans le portefeuille
)

# Afficher les résultats (actions avec un poids > 1%)
print("\nPortefeuille optimal selon Markowitz:")
display(markowitz_portfolio[markowitz_portfolio['weight'] > 0.01].sort_values('weight', ascending=False))

# Extraire et afficher les métriques de performance
performance_metrics = markowitz_portfolio[
    ['expected_return', 'volatility', 'sharpe_ratio', 'max_drawdown', 'effective_assets']
].iloc[0].to_dict()

print("\nMétriques de performance:")
for metric, value in performance_metrics.items():
    if metric in ['expected_return', 'volatility', 'max_drawdown']:
        print(f"{metric.replace('_', ' ').title()}: {value*100:.2f}%")
    elif metric == 'sharpe_ratio':
        print(f"{metric.replace('_', ' ').title()}: {value:.2f}")
    else:
        print(f"{metric.replace('_', ' ').title()}: {value:.2f}")

## Optimisation basée sur les scores du modèle

In [ ]:
# Optimisation basée sur les scores
print("Optimisation basée sur les scores du modèle...")
score_based_portfolio = optimizer.optimize(
    processed_price_data,
    multifactor_scores,
    method="score_based",
    max_weight=0.15,
    min_weight=0.01
)

# Afficher les résultats (actions avec un poids > 1%)
print("\nPortefeuille optimal basé sur les scores:")
display(score_based_portfolio[score_based_portfolio['weight'] > 0.01].sort_values('weight', ascending=False))

# Extraire et afficher les métriques de performance
performance_metrics = score_based_portfolio[
    ['expected_return', 'volatility', 'sharpe_ratio', 'max_drawdown', 'effective_assets']
].iloc[0].to_dict()

print("\nMétriques de performance:")
for metric, value in performance_metrics.items():
    if metric in ['expected_return', 'volatility', 'max_drawdown']:
        print(f"{metric.replace('_', ' ').title()}: {value*100:.2f}%")
    elif metric == 'sharpe_ratio':
        print(f"{metric.replace('_', ' ').title()}: {value:.2f}")
    else:
        print(f"{metric.replace('_', ' ').title()}: {value:.2f}")

## Optimisation par parité de risque

In [ ]:
# Sélectionner les 20 meilleures actions selon le score multifactoriel
top_stocks = multifactor_scores.sort_values('overall_score', ascending=False).head(20)['ticker'].tolist()

# Filtrer les données de prix pour ces actions
top_prices = {ticker: processed_price_data[ticker] for ticker in top_stocks if ticker in processed_price_data}

# Optimisation par parité de risque
print("Optimisation par parité de risque...")
risk_parity_portfolio = optimizer.optimize(
    top_prices,
    multifactor_scores[multifactor_scores['ticker'].isin(top_stocks)],
    method="risk_parity",
    max_weight=0.15,
    min_weight=0.01
)

# Afficher les résultats (actions avec un poids > 1%)
print("\nPortefeuille optimal par parité de risque:")
display(risk_parity_portfolio[risk_parity_portfolio['weight'] > 0.01].sort_values('weight', ascending=False))

# Extraire et afficher les métriques de performance
performance_metrics = risk_parity_portfolio[
    ['expected_return', 'volatility', 'sharpe_ratio', 'max_drawdown', 'effective_assets']
].iloc[0].to_dict()

print("\nMétriques de performance:")
for metric, value in performance_metrics.items():
    if metric in ['expected_return', 'volatility', 'max_drawdown']:
        print(f"{metric.replace('_', ' ').title()}: {value*100:.2f}%")
    elif metric == 'sharpe_ratio':
        print(f"{metric.replace('_', ' ').title()}: {value:.2f}")
    else:
        print(f"{metric.replace('_', ' ').title()}: {value:.2f}")

## Visualisation des différents portefeuilles

In [ ]:
# Comparer les allocations des différents portefeuilles

# Identifier toutes les actions avec un poids significatif dans au moins un portefeuille
markowitz_weights = markowitz_portfolio[markowitz_portfolio['weight'] > 0.01].set_index('ticker')['weight']
score_weights = score_based_portfolio[score_based_portfolio['weight'] > 0.01].set_index('ticker')['weight']
risk_parity_weights = risk_parity_portfolio[risk_parity_portfolio['weight'] > 0.01].set_index('ticker')['weight']

# Combiner tous les tickers
all_tickers = set(markowitz_weights.index) | set(score_weights.index) | set(risk_parity_weights.index)

# Créer un DataFrame pour la comparaison
comparison_df = pd.DataFrame(index=all_tickers)
comparison_df['Markowitz'] = markowitz_weights
comparison_df['Score-Based'] = score_weights
comparison_df['Risk-Parity'] = risk_parity_weights

# Remplacer NaN par 0
comparison_df = comparison_df.fillna(0)

# Trier par la somme des poids
comparison_df['Total'] = comparison_df.sum(axis=1)
comparison_df = comparison_df.sort_values('Total', ascending=False).drop('Total', axis=1)

# Garder uniquement les 15 premières actions
top_comparison = comparison_df.head(15)

# Créer un graphique à barres groupées
plt.figure(figsize=(14, 10))
top_comparison.plot(kind='bar', stacked=False, colormap='viridis')
plt.title('Comparaison des allocations des différents portefeuilles', fontsize=16)
plt.xlabel('Action', fontsize=14)
plt.ylabel('Poids dans le portefeuille', fontsize=14)
plt.xticks(rotation=45, ha='right')
plt.legend(title='Méthode d\'optimisation')
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

## Génération de la frontière efficiente

In [ ]:
# Générer la frontière efficiente pour les 20 meilleures actions
print("Génération de la frontière efficiente...")
efficient_frontier = optimizer.generate_efficient_frontier(
    top_prices,
    top_stocks,
    n_points=30,
    risk_free_rate=0.025,
    max_weight=0.15,
    min_weight=0.01
)

# Afficher la frontière efficiente
plt.figure(figsize=(12, 8))
plt.scatter(
    efficient_frontier['volatility'], 
    efficient_frontier['target_return'], 
    c=efficient_frontier['sharpe_ratio'], 
    cmap='viridis', 
    s=100, 
    alpha=0.8
)

# Ajouter une barre de couleur pour le ratio de Sharpe
cbar = plt.colorbar()
cbar.set_label('Ratio de Sharpe', fontsize=12)

# Marquer les portefeuilles optimisés sur la frontière
plt.scatter(
    [markowitz_portfolio['volatility'].iloc[0]], 
    [markowitz_portfolio['expected_return'].iloc[0]],
    marker='*', s=300, c='red', label='Markowitz (Sharpe optimal)'
)

plt.scatter(
    [score_based_portfolio['volatility'].iloc[0]], 
    [score_based_portfolio['expected_return'].iloc[0]],
    marker='o', s=200, c='green', label='Score-Based'
)

plt.scatter(
    [risk_parity_portfolio['volatility'].iloc[0]], 
    [risk_parity_portfolio['expected_return'].iloc[0]],
    marker='s', s=200, c='orange', label='Risk-Parity'
)

# Ajouter la droite du marché des capitaux (Capital Market Line)
risk_free_rate = 0.025
max_sharpe_idx = efficient_frontier['sharpe_ratio'].idxmax()
max_sharpe_point = (
    efficient_frontier['volatility'].iloc[max_sharpe_idx],
    efficient_frontier['target_return'].iloc[max_sharpe_idx]
)

x_vals = np.linspace(0, max(efficient_frontier['volatility']) * 1.2, 100)
slope = (max_sharpe_point[1] - risk_free_rate) / max_sharpe_point[0]
y_vals = risk_free_rate + slope * x_vals

plt.plot(x_vals, y_vals, 'k--', label='Capital Market Line')

# Ajouter le point du taux sans risque
plt.plot(0, risk_free_rate, 'ro', markersize=10, label='Taux sans risque')

plt.title('Frontière Efficiente des 20 Meilleures Actions', fontsize=16)
plt.xlabel('Volatilité annualisée', fontsize=14)
plt.ylabel('Rendement attendu annualisé', fontsize=14)
plt.grid(True, alpha=0.3)
plt.legend(fontsize=12)
plt.tight_layout()
plt.show()

## Backtest du portefeuille optimisé

In [ ]:
# Sélectionner le portefeuille pour le backtest (ici, nous choisissons le portefeuille basé sur les scores)
selected_portfolio = score_based_portfolio

# Extraire les poids optimaux
weights = dict(zip(
    selected_portfolio['ticker'], 
    selected_portfolio['weight']
))

# Ne garder que les poids significatifs (> 1%)
weights = {ticker: weight for ticker, weight in weights.items() if weight > 0.01}

# Normaliser pour s'assurer que la somme est 1
total_weight = sum(weights.values())
weights = {ticker: weight / total_weight for ticker, weight in weights.items()}

# Réaliser le backtest sur les 2 dernières années
backtest_start = (datetime.now() - timedelta(days=365 * 2)).strftime('%Y-%m-%d')

print(f"Backtest du portefeuille optimisé (basé sur les scores) depuis {backtest_start}...")
backtest_results = optimizer.backtest_portfolio(
    processed_price_data,
    weights,
    start_date=backtest_start,
    rebalance_frequency='M'  # Rééquilibrage mensuel
)

# Afficher les métriques de performance du backtest
backtest_metrics = {
    'Total Return': backtest_results['total_return'].iloc[0],
    'Annualized Return': backtest_results['annualized_return'].iloc[0],
    'Annualized Volatility': backtest_results['annualized_volatility'].iloc[0],
    'Sharpe Ratio': backtest_results['sharpe_ratio'].iloc[0],
    'Maximum Drawdown': backtest_results['max_drawdown'].iloc[0]
}

print("\nRésultats du backtest:")
for metric, value in backtest_metrics.items():
    if metric in ['Total Return', 'Annualized Return', 'Annualized Volatility', 'Maximum Drawdown']:
        print(f"{metric}: {value*100:.2f}%")
    else:
        print(f"{metric}: {value:.2f}")

In [ ]:
# Visualiser la performance du portefeuille vs l'indice de référence
if '^GSPC' in processed_price_data:
    # Extraire les données du S&P 500 pour la période de backtest
    sp500_prices = processed_price_data['^GSPC']['Close']
    sp500_prices = sp500_prices[sp500_prices.index >= backtest_start]
    
    # Calculer la performance normalisée du S&P 500
    sp500_performance = sp500_prices / sp500_prices.iloc[0]
    
    # Préparer les données pour le graphique
    portfolio_performance = backtest_results['portfolio_value'] / backtest_results['portfolio_value'].iloc[0]
    
    # Aligner les dates
    common_dates = backtest_results.index.intersection(sp500_performance.index)
    portfolio_perf_aligned = portfolio_performance.loc[common_dates]
    sp500_perf_aligned = sp500_performance.loc[common_dates]
    
    # Calculer la surperformance
    outperformance = portfolio_perf_aligned - sp500_perf_aligned
    
    # Créer le graphique
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 12), gridspec_kw={'height_ratios': [3, 1]}, sharex=True)
    
    # Performance absolue
    ax1.plot(portfolio_perf_aligned.index, portfolio_perf_aligned, label='Portefeuille optimisé', linewidth=2)
    ax1.plot(sp500_perf_aligned.index, sp500_perf_aligned, label='S&P 500', linewidth=2, alpha=0.7, linestyle='--')
    ax1.set_title('Performance du portefeuille vs S&P 500', fontsize=16)
    ax1.set_ylabel('Performance normalisée', fontsize=14)
    ax1.grid(True, alpha=0.3)
    ax1.legend(fontsize=12)
    
    # Surperformance relative
    ax2.fill_between(
        outperformance.index, 
        outperformance, 
        0, 
        where=(outperformance >= 0), 
        color='green', 
        alpha=0.3, 
        label='Surperformance'
    )
    ax2.fill_between(
        outperformance.index, 
        outperformance, 
        0, 
        where=(outperformance < 0), 
        color='red', 
        alpha=0.3, 
        label='Sous-performance'
    )
    ax2.plot(outperformance.index, outperformance, color='black', linewidth=1)
    ax2.axhline(y=0, color='black', linestyle='-', alpha=0.3)
    ax2.set_title('Surperformance relative vs S&P 500', fontsize=14)
    ax2.set_ylabel('Écart de performance', fontsize=12)
    ax2.set_xlabel('Date', fontsize=12)
    ax2.grid(True, alpha=0.3)
    ax2.legend(fontsize=10)
    
    plt.tight_layout()
    plt.show()
    
    # Calculer les statistiques de surperformance
    max_outperform = outperformance.max()
    min_outperform = outperformance.min()
    avg_outperform = outperformance.mean()
    positive_days = (outperformance > 0).sum()
    total_days = len(outperformance)
    positive_pct = positive_days / total_days * 100 if total_days > 0 else 0
    
    print("\nStatistiques de surperformance:")
    print(f"Maximum: {max_outperform*100:.2f}%")
    print(f"Minimum: {min_outperform*100:.2f}%")
    print(f"Moyenne: {avg_outperform*100:.2f}%")
    print(f"Jours positifs: {positive_days}/{total_days} ({positive_pct:.2f}%)")

## Conclusion et recommandations

Dans ce notebook, nous avons exploré différentes approches d'optimisation de portefeuille en utilisant les scores générés par notre modèle de stock picking :

1. **Optimisation de Markowitz** : Approche classique qui maximise le ratio de Sharpe (rendement ajusté au risque)
2. **Optimisation basée sur les scores** : Approche qui intègre directement les scores multifactoriels du modèle
3. **Optimisation par parité de risque** : Approche qui équilibre la contribution au risque de chaque actif

Les résultats du backtest montrent que notre portefeuille optimisé a (sur)performé le marché (S&P 500) sur la période d'analyse, avec un meilleur rendement ajusté au risque.

### Points clés à retenir

- L'optimisation basée sur les scores permet de surpondérer les actions qui obtiennent les meilleurs scores selon notre modèle multifactoriel
- La parité de risque offre généralement une meilleure diversification et peut être plus robuste dans des marchés volatils
- L'optimisation de Markowitz tend à être plus concentrée sur les actions à faible volatilité et corrélation

### Recommandations

1. **Diversification sectorielle** : Veiller à maintenir une diversification adéquate entre les secteurs
2. **Rééquilibrage périodique** : Procéder à un rééquilibrage mensuel ou trimestriel pour maintenir les allocations cibles
3. **Monitoring continu** : Surveiller régulièrement les scores et ajuster le portefeuille en conséquence
4. **Approche hybride** : Envisager une combinaison des différentes méthodes d'optimisation pour bénéficier de leurs avantages respectifs

Ce notebook démontre comment le modèle de stock picking peut être utilisé pour construire des portefeuilles optimisés qui combinent une analyse fondamentale et technique rigoureuse avec des techniques d'optimisation de portefeuille modernes.